# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import ow_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"City count: {len(cities)}")

City count: 631


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
## Set OpenWeather API base url
ow_url = "http://api.openweathermap.org/data/2.5/weather?q="
## Set Units
units = "imperial"

## Create weather data empty list
weather_data = []
## Set initial values
record = 1

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for city in cities:
        ##
        query_url = f"{ow_url}{city}&appid={ow_key}&units={units}"
        response = requests.get(query_url).json()
        try:
            ## Print Log Status
            print(f"Processing Record {record:>2d} | City Id = {response['id']:<7d} [{city}]")
            ## Retrieve weather data from OpenWeather API (response)
            weather_data.append({
                'City': response['name'],
                'Lat': response['coord']['lat'],
                'Lng': response['coord']['lon'],
                'Max Temp': response['main']['temp_max'],
                'Humidity': response['main']['humidity'],
                'Cloudiness': response['clouds']['all'],
                'Wind Speed': response['wind']['speed'],
                'Country': response['sys']['country'],
                'Date': response['dt']
            })
            ## Record counter
            record += 1
        except KeyError:
            ## Print Log Status
            print(f"City [{city}] not found. Skipping...")
            pass

## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Record  1 | City Id = 3831208 [qaanaaq]
Processing Record  2 | City Id = 2126710 [beringovskiy]
Processing Record  3 | City Id = 1015776 [bredasdorp]
Processing Record  4 | City Id = 3401845 [conceicao do araguaia]
Processing Record  5 | City Id = 4036284 [alofi]
Processing Record  6 | City Id = 3437443 [pozo colorado]
Processing Record  7 | City Id = 5880054 [barrow]
Processing Record  8 | City Id = 5128316 [nassau]
Processing Record  9 | City Id = 3443061 [chuy]
Processing Record 10 | City Id = 2295021 [sunyani]
Processing Record 11 | City Id = 2163355 [hobart]
Processing Record 12 | City Id = 3474574 [palmas]
Processing Record 13 | City Id = 3421765 [nanortalik]
Processing Record 14 | City Id = 2075265 [busselton]
Processing Record 15 | City Id = 3526617 [jalapa]
Processing Record 16 | City Id = 3874787 [punta arenas]
Processing Record 17 | City Id = 1633419 [padang]
Processing Record 18 | City Id = 5380437 [pacific 

Processing Record 147 | City Id = 1853174 [sakaiminato]
Processing Record 148 | City Id = 6165406 [thompson]
Processing Record 149 | City Id = 5554428 [ketchikan]
City [karauzyak] not found. Skipping...
Processing Record 150 | City Id = 3617154 [puerto cabezas]
Processing Record 151 | City Id = 3520989 [puerto madero]
Processing Record 152 | City Id = 556268  [ostrovnoy]
Processing Record 153 | City Id = 1688696 [san policarpo]
Processing Record 154 | City Id = 3377301 [mabaruma]
Processing Record 155 | City Id = 728674  [obzor]
Processing Record 156 | City Id = 1268015 [kannangad]
Processing Record 157 | City Id = 3141667 [roald]
Processing Record 158 | City Id = 2125693 [evensk]
Processing Record 159 | City Id = 935215  [saint-philippe]
Processing Record 160 | City Id = 1052373 [beira]
Processing Record 161 | City Id = 3450376 [santa cecilia]
Processing Record 162 | City Id = 2178753 [kirakira]
City [amderma] not found. Skipping...
Processing Record 163 | City Id = 473263  [vizinga]


Processing Record 295 | City Id = 5563839 [fortuna]
Processing Record 296 | City Id = 2206874 [waipawa]
Processing Record 297 | City Id = 3625710 [upata]
Processing Record 298 | City Id = 3996737 [maneadero]
Processing Record 299 | City Id = 2742416 [barcelos]
Processing Record 300 | City Id = 60019   [eyl]
City [halalo] not found. Skipping...
Processing Record 301 | City Id = 1214026 [sabang]
City [attawapiskat] not found. Skipping...
Processing Record 302 | City Id = 2137773 [vao]
Processing Record 303 | City Id = 2561668 [agadir]
Processing Record 304 | City Id = 1270612 [guru har sahai]
Processing Record 305 | City Id = 656820  [iisalmi]
Processing Record 306 | City Id = 2514301 [mao]
Processing Record 307 | City Id = 2205631 [raymond terrace]
Processing Record 308 | City Id = 1264976 [leh]
Processing Record 309 | City Id = 1152562 [kui buri]
Processing Record 310 | City Id = 1490256 [talnakh]
Processing Record 311 | City Id = 1863997 [fukue]
Processing Record 312 | City Id = 34489

Processing Record 440 | City Id = 3452233 [propria]
Processing Record 441 | City Id = 5072006 [lincoln]
Processing Record 442 | City Id = 3120431 [igualada]
Processing Record 443 | City Id = 3175096 [lasa]
Processing Record 444 | City Id = 485064  [syumsi]
Processing Record 445 | City Id = 3664321 [eirunepe]
Processing Record 446 | City Id = 1337610 [thinadhoo]
Processing Record 447 | City Id = 1490364 [tabory]
Processing Record 448 | City Id = 2120048 [ust-nera]
Processing Record 449 | City Id = 3146487 [raudeberg]
Processing Record 450 | City Id = 113659  [tabas]
Processing Record 451 | City Id = 962367  [richards bay]
Processing Record 452 | City Id = 336350  [goba]
Processing Record 453 | City Id = 935051  [lavumisa]
Processing Record 454 | City Id = 1812256 [dongsheng]
City [artyk] not found. Skipping...
Processing Record 455 | City Id = 3389822 [salinopolis]
Processing Record 456 | City Id = 2155562 [nelson bay]
City [mullaitivu] not found. Skipping...
Processing Record 457 | Cit

Processing Record 583 | City Id = 4947459 [pittsfield]
Processing Record 584 | City Id = 10722858 [svetlogorsk]
------------------------------
Data Retrieval Complete
------------------------------


In [4]:
# Print the city count to confirm updated count
print(f"Updated city count: {len(weather_data)}")

Updated city count: 584


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [18]:
## Build DataFrame
raw_city_data = pd.DataFrame(data = weather_data)

## Export DataFrame (of all retrieved data) to CSV
raw_city_data.to_csv('Output/Cities.csv', index=False, header=True, encoding='utf-8')

## Display DataFrame
raw_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaanaaq,77.4840,-69.3632,39.56,78,100,9.04,GL,1619539622
1,Beringovskiy,63.0500,179.3167,30.43,98,100,4.52,RU,1619539622
2,Bredasdorp,-34.5322,20.0403,66.20,63,98,9.22,ZA,1619539622
3,Conceição do Araguaia,-8.2578,-49.2647,80.60,78,40,8.05,BR,1619539623
4,Alofi,-19.0595,-169.9187,82.15,77,100,22.41,NU,1619539623


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [27]:
## Inspect data to check max humidity is equal or less than 100%
raw_city_data.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,5.840000e+02
mean,19.988692,14.860307,62.458682,66.113014,54.109589,7.986096,1.619540e+09
std,33.335813,90.111101,20.997337,23.055922,37.436899,4.829381,9.469462e+01
min,-54.800000,-175.200000,-3.190000,5.000000,0.000000,0.360000,1.619539e+09
25%,-8.161950,-67.114250,46.400000,52.000000,20.000000,4.335000,1.619540e+09
50%,24.910450,20.598850,66.400000,70.000000,68.000000,6.910000,1.619540e+09
75%,48.603525,95.217825,78.800000,85.000000,90.000000,10.695000,1.619540e+09
max,78.218600,179.316700,107.470000,100.000000,100.000000,24.160000,1.619540e+09


In [37]:
#  Get the indices of cities that have humidity over 100%.
outlier_index = raw_city_data.loc[raw_city_data['Humidity']>100].index
print(f"Number of cities with humidity over 100%: {len(outlier_index)}")

Number of cities with humidity over 100%: 0


In [33]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = raw_city_data.drop(index=outlier_index, inplace=False)
clean_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaanaaq,77.4840,-69.3632,39.56,78,100,9.04,GL,1619539622
1,Beringovskiy,63.0500,179.3167,30.43,98,100,4.52,RU,1619539622
2,Bredasdorp,-34.5322,20.0403,66.20,63,98,9.22,ZA,1619539622
3,Conceição do Araguaia,-8.2578,-49.2647,80.60,78,40,8.05,BR,1619539623
4,Alofi,-19.0595,-169.9187,82.15,77,100,22.41,NU,1619539623


In [39]:
## Inspect data to check max humidity is equal or less than 100%
clean_city_data.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,5.840000e+02
mean,19.988692,14.860307,62.458682,66.113014,54.109589,7.986096,1.619540e+09
std,33.335813,90.111101,20.997337,23.055922,37.436899,4.829381,9.469462e+01
min,-54.800000,-175.200000,-3.190000,5.000000,0.000000,0.360000,1.619539e+09
25%,-8.161950,-67.114250,46.400000,52.000000,20.000000,4.335000,1.619540e+09
50%,24.910450,20.598850,66.400000,70.000000,68.000000,6.910000,1.619540e+09
75%,48.603525,95.217825,78.800000,85.000000,90.000000,10.695000,1.619540e+09
max,78.218600,179.316700,107.470000,100.000000,100.000000,24.160000,1.619540e+09


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression